In [1]:
import numpy as np
import pandas as pd
import baostock as bs

a | b|
:---:|:---:|
code | 股票代码，sh或sz.+6位数字代码，或者指数代码，如：sh.601398。sh：上海；sz：深圳。此参数不可为空
 fields | 指示简称，支持多指标输入，以半角逗号分隔，填写内容作为返回类型的列。详细指标列表见历史行情指标参数章节，日线与分钟线参数不同。此参数不可为空；
 start | 开始日期（包含），格式“YYYY-MM-DD”，为空时取2015-01-01；
 end | 结束日期（包含），格式“YYYY-MM-DD”，为空时取最近一个交易日；
 frequency | 数据类型，默认为d，日k线；d=日k线、w=周、m=月、5=5分钟、15=15分钟、30=30分钟、60=60分钟k线数据，不区分大小写；指数没有分钟线数据；周线每周最后一个交易日才可以获取，月线每月最后一个交易日才可以获取。
 adjustflag | 复权类型，默认不复权：3；1：后复权；2：前复权。已支持分钟线、日线、周线、月线前后复权。


| 参数名称   |    参数参数描述 | 算法说明 |
|:---------:|:-------------:|:-------:|
|date |	交易所行情日期 | -|	
code |	证券代码 | - |	
open |	开盘价	| - |
high |	最高价	|- |
low	| 最低价 | |	
close	| 收盘价 | |	
preclose |	前收盘价 |	见表格下方详细说明
volume	| 成交量 |（累计 单位：股）	
amount	| 成交额 |（单位：人民币元）	
adjustflag |	复权状态(1：后复权， 2：前复权，3：不复权）	| |
turn	| 换手率 |	[指定交易日的成交量(股)/指定交易日的股票的流通股总股数(股)]*100%
tradestatus |	交易状态(1：正常交易 0：停牌）	
pctChg	 | 涨跌幅 |（百分比）	日涨跌幅=[(指定交易日的收盘价-指定交易日前收盘价)/指定交易日前收盘价]*100%
peTTM |	滚动市盈率	| (指定交易日的股票收盘价/指定交易日的每股盈余TTM)=(指定交易日的股票收盘价*截至当日公司总股本)/归属母公司股东净利润TTM
pbMRQ |	市净率	 | (指定交易日的股票收盘价/指定交易日的每股净资产)=总市值/(最近披露的归属母公司股东的权益-其他权益工具)
psTTM |	滚动市销率	| (指定交易日的股票收盘价/指定交易日的每股销售额)=(指定交易日的股票收盘价*截至当日公司总股本)/营业总收入TTM
pcfNcfTTM |	滚动市现率 |	(指定交易日的股票收盘价/指定交易日的每股现金流TTM)=(指定交易日的股票收盘价*截至当日公司总股本)/现金以及现金等价物净增加额TTM
isST  | 	是否ST股，1是，0否

In [2]:
bs.login()

login success!


##### single ticker query

In [299]:
rs = bs.query_history_k_data_plus(
    code='sz.300073', \
    fields="date,code,\
        open,high,low,close, preclose, \
        volume, amount,\
        adjustflag, \
        turn, \
        tradestatus, \
        pctChg, \
        isST",
    start_date='2020-08-20', \
    end_date='2020-08-31', \
    frequency="d", \
    adjustflag="3")

rs.get_data()

,date,code,open,high,low,close,preclose,volume,amount,adjustflag,turn,tradestatus,pctChg,isST
0,2020-08-20,sz.300073,38.7900,39.4800,37.9200,38.7700,38.7900,9357190,361468577.2800,3,2.175400,1,-0.051600,0
1,2020-08-21,sz.300073,39.3000,39.4200,38.4300,39.2600,38.7700,9648680,375879858.7700,3,2.243200,1,1.263900,0
2,2020-08-24,sz.300073,39.5000,39.8600,37.4100,39.2900,39.2600,12093149,471434735.6600,3,2.811500,1,0.076400,0
3,2020-08-25,sz.300073,39.3600,42.3300,38.8000,40.5300,39.2900,19273736,781679401.3300,3,4.476900,1,3.156000,0
4,2020-08-26,sz.300073,40.7800,40.7800,37.5000,38.0200,40.5300,16932659,654912858.3700,3,3.933200,1,-6.192900,0
5,2020-08-27,sz.300073,38.4700,39.3000,37.3500,37.8300,38.0200,9797106,373058884.7300,3,2.275700,1,-0.499700,0
6,2020-08-28,sz.300073,37.8300,37.8300,37.8300,37.8300,37.8300,0,0.0000,3,,0,0.000000,0
7,2020-08-31,sz.300073,37.8300,37.8300,37.8300,37.8300,37.8300,0,0.0000,3,,0,0.000000,0


In [4]:
with open('./stock_names.txt', 'r') as f:
    tickers = [i.strip('\n') for i in f.readlines()]

In [5]:
def trans_tick(tick):
    return tick[:2] + '.' + tick[2:]

In [7]:
bs.login()
start ='2020-08-20'
end ='2020-08-31'
turnover_th = 5
# composite = 'smeboard' # chuangyeban mainboard主板, smeboard, small and medium enterprise
#composite = 'mainboard'
composite = 'gemboard'
raw_data = []
gem_lst = []
rm_lst = []

if composite == 'gemboard':
    prefix = 'sz.300'
elif composite == 'smeboard':
    prefix = 'sz.00'
elif composite == 'mainboard':
    prefix = 'sh.60'
else:
    raise NotImplementedError

# chuangyeban index
rs = bs.query_history_k_data_plus(
        code='sz.399006', \
        fields="date,code,\
            open,high,low,close, preclose, \
            volume, amount,\
            adjustflag, \
            turn, \
            tradestatus, \
            pctChg, \
            isST",
        start_date=start, \
        end_date=end, \
        frequency="d", \
        adjustflag="3")
gem_index_span = 0
while rs.next():
    rs.get_row_data()
    gem_index_span += 1
    
resign_flag = True
for ind, tick in enumerate(tickers):
    tick = trans_tick(tick)
    if tick.startswith(prefix):
        single_tick_data = []
        rs = bs.query_history_k_data_plus(
            code=tick, \
            fields="date,code,\
                open,high,low,close, preclose, \
                volume, amount,\
                adjustflag, \
                turn, \
                tradestatus, \
                pctChg, \
                isST",
            start_date=start, \
            end_date=end, \
            frequency="d", \
            adjustflag="3")
        while rs.next():
            single_tick_data.append(rs.get_row_data())
            
        r2dim_np = np.expand_dims(np.array(single_tick_data), 0)
        if len(r2dim_np.shape) < 3: # 开始计算日期之前退市
            continue
        if r2dim_np.shape[1] < gem_index_span: # 开始计算日期之后退市
            continue
        try:
            # 停牌换手率数据为''
            r2dim_np[:, :, 10] = r2dim_np[:, :, 10].astype(np.float32)
        except:
            continue
            
        if resign_flag:
            raw_np = r2dim_np
            resign_flag = False
        else:
            raw_np = np.concatenate((raw_np, r2dim_np), axis=0)
        gem_lst.append(tick)

login success!


In [8]:
turnover_np = raw_np[:, :, 10].astype(np.float32)
turnover_np_gain = (turnover_np[:, 1:] - turnover_np[:, 0:-1]) / turnover_np[:, 0:-1]
proper_tick_index = np.max(turnover_np_gain, 1) > turnover_th
gem_lst_np = np.array(gem_lst)
gem_lst_np[proper_tick_index]

array(['sz.300216', 'sz.300245', 'sz.300609', 'sz.300789'], dtype='<U9')